<a href="https://colab.research.google.com/github/tranbaquang1708/GraduationThesis/blob/develop/rbf_sphere_comparetime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [47]:
# Time comparation between vectorized code and nonvectorized code
# Dataset: Sphere

In [48]:
import sys
import os
mp = os.path.abspath(os.path.join('/content/drive/My Drive/GraduationThesis/gitrepo/GraduationThesis/'))
sys.path.append(mp)

In [49]:
import numpy as np
import timeit
from modules import Rbf

In [50]:
# Import point data
filename = mp + "/dataset/3d/sphere.xyz"
dataset,normal_vectors = Rbf.gen_from_txt3(filename, offset=0.01)

In [51]:
SETUP = """
import numpy as np
from numpy import ma
from scipy import linalg
from scipy.spatial.distance import squareform,pdist
import scipy
from __main__ import dataset
"""

In [52]:
# Non-vectorized version of solving weight function and display memory usage
LOOP_SOLVE = """
offset = 0.01
num_points = len(dataset)
num_on_points = int(num_points/3)

A = np.zeros((num_points, num_points))
for i in range(num_points):
  for j in range(num_points):
    norm = (dataset[i,0] - dataset[j,0])**2 + (dataset[i,1] - dataset[j,1])**2 + (dataset[i,2] - dataset[j,2])**2
    if norm != 0:
      A[i,j] = np.sqrt(norm)

print("Data: " + str(num_on_points) + " points")
print("Memory usage: " + str(sys.getsizeof(A)) + " bytes")
print()

## b: the vector of the function values at the point from the data-set
b = np.zeros(num_points)

for i in range(num_on_points, 2*num_on_points):
  b[i] = -offset

for i in range(2*num_on_points,3*num_on_points):
  b[i] = offset

w = scipy.linalg.solve(A,b)
"""

In [53]:
# Vectorized version of solving weight function 
VEC_SOLVE= """
offset = 0.01
num_points = len(dataset)
num_on_points = int(num_points/3)

r = squareform(pdist(dataset))
A = r

## b: the vector of the function values at the point from the data-set
b = np.zeros(num_points)

b[num_on_points:2*num_on_points] = -offset
b[2*num_on_points:3*num_on_points] = offset

w = scipy.linalg.solve(A,b)
"""

In [54]:
# Time comparison
loop_time = timeit.timeit(LOOP_SOLVE, SETUP, number = 1)
print("Non-vectorized: " + str(loop_time) + " second(s)")

vec_time = timeit.timeit(setup = SETUP, stmt = VEC_SOLVE, number = 1)
print("Vectorized: " + str(vec_time) + " second(s)")

print("Speedup: " + str(loop_time/vec_time) + " times")

Data: 642 points
Memory usage: 29675920 bytes

Non-vectorized: 16.68093865500009 second(s)
Vectorized: 0.2946086970000579 second(s)
Speedup: 56.62065928419219 times


In [55]:
# Solve weight function
w = Rbf.solve_w_func(dataset, offset=0.01, func=Rbf.thin_plate)

In [56]:
SETUP = """
import numpy as np
from numpy import ma
from scipy import linalg
from scipy.spatial.distance import squareform,pdist
import scipy
from __main__ import dataset,xx,yy,zz,w
"""

In [57]:
# Non-vectorized version of sampling function
LOOP_SAMPLING = """
dimg = xx.shape
dimw = w.shape
z = np.zeros(dimg)
for i in range(dimg[0]):
  for j in range(dimg[1]):
    for l in range(dimg[2]):
      z[i,j] = 0.0
      for k in range(dimw[0]):
        nxy = (xx[i,j,l]-dataset[k,0])**2 + (yy[i,j,l]-dataset[k,1])**2 + (zz[i,j,l]-dataset[k,2])**2
        if (nxy != 0):
          z[i,j] += w[k]*nxy
"""

In [58]:
# Vectorized version of sampling function
VEC_SAMPLING = """
dimg = xx.shape
dimw = w.shape
z = np.zeros(dimg)
for k in range(int(dimw[0])):
  r = (xx - dataset[k,0])**2 + (yy - dataset[k,1])**2 + (zz - dataset[k,2])**2
  z += w[k] * np.sqrt(r)
"""

In [59]:
# Create 2D regular grid obtained
xx,yy,zz = Rbf.grid(dataset[:,0], dataset[:,1], dataset[:,2])

In [60]:
# Time comparison
loop_time = timeit.timeit(LOOP_SAMPLING, SETUP, number=1)
print("Non-vectorized: " + str(loop_time) + " second(s)")

vec_time = timeit.timeit(VEC_SAMPLING, SETUP, number=1)
print("Vectorized: " + str(vec_time) + " second(s)")

print("Speedup: " + str(loop_time/vec_time) + " times")

Non-vectorized: 1347.5872300869999 second(s)
Vectorized: 3.0901942860000418 second(s)
Speedup: 436.08495303747435 times
